#DATASCI W261: Machine Learning at Scale

#Assignment: Week 2

- Juanjo Carin
- [juanjose.carin@ischool.berkeley.edu](mailto:juanjose.carin@ischol.berkeley.com)
- W261-2
- Week 02
- Submission date: 9/15/2015

In [5]:
# Start Hadoop
!/usr/local/hadoop/sbin/start-yarn.sh
!/usr/local/hadoop/sbin/start-dfs.sh

starting yarn daemons
starting resourcemanager, logging to /usr/local/hadoop/logs/yarn-hduser-resourcemanager-juanjo-VB.out
localhost: starting nodemanager, logging to /usr/local/hadoop/logs/yarn-hduser-nodemanager-juanjo-VB.out
15/09/19 17:17:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Starting namenodes on [localhost]
localhost: starting namenode, logging to /usr/local/hadoop/logs/hadoop-hduser-namenode-juanjo-VB.out
localhost: starting datanode, logging to /usr/local/hadoop/logs/hadoop-hduser-datanode-juanjo-VB.out
Starting secondary namenodes [0.0.0.0]
0.0.0.0: starting secondarynamenode, logging to /usr/local/hadoop/logs/hadoop-hduser-secondarynamenode-juanjo-VB.out
15/09/19 17:18:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [1]:
# Create new folder
!hdfs dfs -mkdir -p /user/hadoop/dirhw2

15/09/20 10:14:13 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
## Upload input file to HDFS
!hdfs dfs -put -f enronemail_1h.txt /user/hadoop/dirhw2

15/09/20 10:14:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


#HW2.5

**Using the Enron data from HW1 an in the  Hadoop MapReduce framework, write  a mapper/reducer for a multinomial Naive Bayes Classifier that will classify the email messages using words present. Also drop words with a frequency of less than three (3). How does it affect the misclassifcation error of learnt naive multinomial Bayesian Classifiers on the training dataset:**

The first stage is a MapReduce job that learns the whole vocabulary from the training set. The output (a dictionary with all the words present) will be used in the second stage.

In [3]:
%%writefile mapper1.py
#!/usr/bin/python
import sys
import re
import os

vocabulary = []

# input comes from STDIN (standard input)
for line in sys.stdin:
    content = ' '.join(line.strip().split("\t")[2:])
    # We search the word in both the subject and the content
        # because one or the other may not exist, but the way the data are
        # stored we don't know which one may be missing
    content = re.sub('[^a-z]', ' ', content.lower())
    # Discard non-alphanumeric characters and also numbers
    words = content.split() # extract words
    words = set(words) # extract unique words
    vocabulary[1:1] = words # append to vocabulary
for word in set(vocabulary):
    print '%s\t%s' % (word, 1) # value here is not important

Overwriting mapper1.py


In [4]:
%%writefile reducer1.py
#!/usr/bin/python
import sys

vocabulary = []
for line in sys.stdin:
    # Take key only (the word) and add to vocabulary if not present
    word = line.split("\t")[0]
    #if word not in vocabulary:
    #    print word
    # If we use the 2 lines above instead of the 3 lines below
        # each word in the vocabulary goes in a new line
        # (and there's no need to sort)
    vocabulary.append(word)
vocabulary = sorted(set(vocabulary)) # Get unique words
print ' '.join(vocabulary) # Print words separated by space

Overwriting reducer1.py


In [5]:
# Delete previous output (if it exists)
!hdfs dfs -rm -r dictionary

15/09/20 10:14:30 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/20 10:14:32 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted dictionary


In [6]:
# Hadoop streaming command
    # Forcing number of reducers to be 1
!hadoop jar /home/hduser/Downloads/hadoop-streaming*.jar -D \
    mapred.reduce.tasks=1 -mapper mapper1.py -reducer reducer1.py -input \
    /user/hadoop/dirhw2/enronemail_1h.txt -output dictionary

15/09/20 10:14:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/20 10:14:37 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/09/20 10:14:37 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/09/20 10:14:37 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/09/20 10:14:37 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/20 10:14:38 INFO mapreduce.JobSubmitter: number of splits:1
15/09/20 10:14:38 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
15/09/20 10:14:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1585235232_0001
15/09/20 10:14:39 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
15/09/20 10:14:39 INFO mapreduce.Job: Running job: jo

In [7]:
# Move output to local
# Delete it from local if a previous version exists
!rm ~/Downloads/HW2/dictionary.txt
!hdfs dfs -copyToLocal dictionary/part-00000 ~/Downloads/HW2/dictionary.txt

15/09/20 10:14:48 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The dictionay we've created have the following words on it:

`a ab abidjan ability able abn about above absent absenteeism absolute absolutely absorb abuse abused acce accelerate accelerated accept acceptable accepted accepting accepts access accomodate accomodates accompanied according accordingly account accountability accounting accounts accrual accurate aches achieve achieved acid acquire acquisition acrobaat acrobat across act action activate active activists activities actor actress actual actually ad adage adams adapted add added adding addition additional additionally address addressed addresses addressing addtional adequately adhesion adm admin adminder administration admitted admixture adobe adobee adolescent adr adrianbold ads adult adv advance advanced advantage ...`

The second stage is a MapReduce job that applies the vocabulary to build the classifier. A parameter (`min_occurr`) is used to drop those words with a frequence less than that number.

In [8]:
!hdfs dfs -put -f ~/Downloads/HW2/dictionary.txt

15/09/20 10:15:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
%%writefile mapper2.py
#!/usr/bin/python
import sys
import re

f = open('dictionary', 'r')
word_dict = []
for line in f:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    words = line.split()
    for word in words:
        word_dict.append(word)

# input comes from STDIN (standard input)
for line in sys.stdin:
    word_count = 0 # count of word in the email
    ID = line.split("\t")[0]
    TRUTH = line.split("\t")[1]
    content = ' '.join(line.strip().split("\t")[2:])
        # We search the word in both the subject and the content
            # because one or the other may not exist, but the way the data are
            # stored we don't know which one may be missing
    content = re.sub('[^a-z]', ' ', content.lower())
    words = content.split() # extract words
    for word in set(word_dict):
        print word + '\t' + str(words.count(word)) + '\t' + ID + '\t' + TRUTH

Overwriting mapper2.py


In [10]:
%%writefile reducer2.py
#!/home/hduser/anaconda/bin/python
import sys
import os
from math import log
import numpy as np

# Word to use find/use
env_vars = os.environ
min_occurr = env_vars['min_occurr']

word = []
word_count = []
ID = []
TRUTH = []

for line in sys.stdin:
    key_values = line.split('\t')
    # The key is ONE of the words we're counting
    if len(word) != 0:
        if word[-1] != key_values[0]:
            word.append(key_values[0])
    else:
        word.append(key_values[0])
    word_count.append(int(key_values[1]))
    # ID and TRUTH are replicated for each word in the vocabulary
        # so we just need to keep track of them once
    if len(word) == 1:
        ID.append(key_values[2])
        TRUTH.append(int(key_values[3]))

# The lists above will have a length equal to
    # number_different_words_in_vocab * number_emails_in_dataset
vocab_size = len(word)
num_emails = len(TRUTH)
# Reshape the list word_count into a 2-D numpy array
word_count = np.array(word_count).reshape(len(word), num_emails)
# Drop words with a frequency of less than min_ocur
condition = np.sum(word_count,1) >= int(min_occurr)
final_word_count = word_count[condition, :]
filtered_indices = np.extract(condition, word_count).tolist()
final_word = [word[i] for i in filtered_indices]
final_vocab_size = len(final_word)
                        
total_spam = sum(TRUTH) # total count of spam emails
total_ham = len(TRUTH) - total_spam # total count of ham emails
total_word_spam = [0]*len(final_word)
total_word_ham = [0]*len(final_word)
for i,w in enumerate(final_word):
    # Total count of word w in spam emails
    total_word_spam[i] = sum([x*y for (x,y) in zip(TRUTH,final_word_count[i])]) 
    # Total count of word w in ham emails
    total_word_ham[i] = sum(final_word_count[i]) - total_word_spam[i]

# PRIORS
prob_ham = float(total_ham)/(total_ham+total_spam)
prob_spam = 1 - prob_ham
# CONDITIONAL LIKELIHOODS
prob_word_ham = [float(x+1) / (sum(total_word_ham)+final_vocab_size) for x \
                 in total_word_ham]
prob_word_spam = [float(x+1) / (sum(total_word_spam)+final_vocab_size) for \
                  x in total_word_spam]

# Assess classification with the training set 
CLASS = [0]*num_emails
for i in range(num_emails): # for each email
    # POSTERIORS
    prob_ham_word = log(prob_ham,10) + \
        sum([x*log(y,10) for (x,y) in zip(final_word_count[:,i],prob_word_ham)])
    prob_spam_word = log(prob_spam,10) + \
        sum([x*log(y,10) for (x,y) in zip(final_word_count[:,i],prob_word_spam)])
    # The right side of the equations are not equal to prob_category_word, but 
        # to log(prob_category_word) - log(prob_word) (where prob_word is the 
        # EVIDENCE). It's OK since we only want to compare the POSTERIORS
    if prob_spam_word > prob_ham_word: # classify as spam if posterior is higher
        CLASS[i] = 1
    # Output for each email
    print ID[i] + '\t' + str(TRUTH[i]) + '\t' + str(CLASS[i])

# Training error
# Count of misclassification errors
errors = sum([x!=y for (x,y) in zip(TRUTH,CLASS)])
training_error = float(errors) / len(TRUTH)
# Additional line
print 'Training Error\t' + str(training_error)

Overwriting reducer2.py


In [11]:
# Delete previous output (if it exists)
!hdfs dfs -rm -r allNBEnron

15/09/20 10:15:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/20 10:15:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted allNBEnron


First we try with `min_occurr=1`, i.e., using all words, regardless of their frequency.

In [12]:
# Hadoop streaming command
!hadoop jar /home/hduser/Downloads/hadoop-streaming*.jar -D \
    mapred.reduce.tasks=1 -files 'dictionary.txt#dictionary' -cmdenv \
    min_occurr=1  -mapper mapper2.py -reducer reducer2.py -input \
    /user/hadoop/dirhw2/enronemail_1h.txt -output allNBEnron

15/09/20 10:15:10 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/20 10:15:11 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
15/09/20 10:15:12 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/09/20 10:15:12 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/09/20 10:15:12 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/09/20 10:15:13 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/20 10:15:13 INFO mapreduce.JobSubmitter: number of splits:1
15/09/20 10:15:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1243911755_0001
15/09/20 10:15:15 INFO mapred.LocalDistributedCacheManager: Creating symlink: /app/hadoop/tmp/mapred/local/1442769314573/dictionary.txt <- /

In [13]:
!hdfs dfs -cat allNBEnron/part-00000

15/09/20 10:15:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
0016.2001-07-06.SA_and_HP	1	1
0018.2001-07-13.SA_and_HP	1	1
0012.2000-01-17.beck	0	0
0012.2000-06-08.lokay	0	0
0006.2001-02-08.kitchen	0	0
0017.2000-01-17.beck	0	0
0010.2001-06-28.SA_and_HP	1	1
0007.1999-12-14.farmer	0	0
0002.1999-12-13.farmer	0	0
0003.2000-01-17.beck	0	0
0008.2003-12-18.GP	1	1
0007.1999-12-13.kaminski	0	0
0016.1999-12-15.farmer	0	0
0004.2004-08-01.BG	1	1
0011.2001-06-28.SA_and_HP	1	1
0001.2000-06-06.lokay	0	0
0002.2003-12-18.GP	1	1
0004.1999-12-10.kaminski	0	0
0012.2001-02-09.kitchen	0	0
0002.2004-08-01.BG	1	1
0004.2001-04-02.williams	0	0
0003.2003-12-18.GP	1	1
0010.1999-12-14.farmer	0	0
0005.2000-06-06.lokay	0	0
0013.2001-06-30.SA_and_HP	1	1
0007.2000-01-17.beck	0	0
0006.2001-04-03.williams	0	0
0005.2001-02-08.kitchen	0	0
0001.2001-02-07.kitchen	0	0
0011.2004-08-01.BG	1	1
0009.1999-12-13.kaminski	0	0
0016.2004-08-01.BG	1	

As seen above (and as expected) the training error is **zero**, so the accuracy at classifying the same dataset used to train the model is perfect. Now let's drop all words with a frequency of less than three.

In [14]:
# Delete previous output (if it exists)
!hdfs dfs -rm -r allNBEnron

15/09/20 10:15:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/20 10:16:01 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted allNBEnron


In [15]:
# Hadoop streaming command
!hadoop jar /home/hduser/Downloads/hadoop-streaming*.jar -D \
    mapred.reduce.tasks=1 -files 'dictionary.txt#dictionary' -cmdenv \
    min_occurr=3  -mapper mapper2.py -reducer reducer2.py -input \
    /user/hadoop/dirhw21/enronemail_1h.txt -output allNBEnron

15/09/20 10:16:04 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
15/09/20 10:16:06 INFO Configuration.deprecation: mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
15/09/20 10:16:08 INFO Configuration.deprecation: session.id is deprecated. Instead, use dfs.metrics.session-id
15/09/20 10:16:08 INFO jvm.JvmMetrics: Initializing JVM Metrics with processName=JobTracker, sessionId=
15/09/20 10:16:08 INFO jvm.JvmMetrics: Cannot initialize JVM Metrics with processName=JobTracker, sessionId= - already initialized
15/09/20 10:16:09 INFO mapred.FileInputFormat: Total input paths to process : 1
15/09/20 10:16:09 INFO mapreduce.JobSubmitter: number of splits:1
15/09/20 10:16:10 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local524957098_0001
15/09/20 10:16:11 INFO mapred.LocalDistributedCacheManager: Creating symlink: /app/hadoop/tmp/mapred/local/1442769371337/dictionary.txt <- /h

In [16]:
!hdfs dfs -cat allNBEnron/part-00000

15/09/20 10:16:35 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
0016.2001-07-06.SA_and_HP	1	1
0018.2001-07-13.SA_and_HP	1	1
0012.2000-01-17.beck	0	0
0012.2000-06-08.lokay	0	0
0006.2001-02-08.kitchen	0	0
0017.2000-01-17.beck	0	0
0010.2001-06-28.SA_and_HP	1	1
0007.1999-12-14.farmer	0	0
0002.1999-12-13.farmer	0	0
0003.2000-01-17.beck	0	0
0008.2003-12-18.GP	1	1
0007.1999-12-13.kaminski	0	0
0016.1999-12-15.farmer	0	0
0004.2004-08-01.BG	1	1
0011.2001-06-28.SA_and_HP	1	1
0001.2000-06-06.lokay	0	0
0002.2003-12-18.GP	1	1
0004.1999-12-10.kaminski	0	0
0012.2001-02-09.kitchen	0	0
0002.2004-08-01.BG	1	1
0004.2001-04-02.williams	0	0
0003.2003-12-18.GP	1	1
0010.1999-12-14.farmer	0	0
0005.2000-06-06.lokay	0	0
0013.2001-06-30.SA_and_HP	1	1
0007.2000-01-17.beck	0	0
0006.2001-04-03.williams	0	0
0005.2001-02-08.kitchen	0	0
0001.2001-02-07.kitchen	0	0
0011.2004-08-01.BG	1	1
0009.1999-12-13.kaminski	0	0
0016.2004-08-01.BG	1	

Dropping very infrequent words causes the training error to be just slightly higher (**1%** instead of 0%). If we try higher values of `min_occurr` (dropping words unless they are quite frequent), the training error keeps increasing, though at a very low rate.

At least in a real case (measuring accuracy with a test set rather than the training set), it makes more sense to use very frequent words as **stopwords** (i.e., to drop words that appear more than $n$ times), because they're likely to be present in both types of emails (spam and ham), and hence do not characterize the kind of email we're trying to classify.

In [45]:
!/usr/local/hadoop/sbin/stop-yarn.sh
!/usr/local/hadoop/sbin/stop-dfs.sh

stopping yarn daemons
stopping resourcemanager
localhost: stopping nodemanager
no proxyserver to stop
15/09/19 17:26:46 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Stopping namenodes on [localhost]
localhost: stopping namenode
localhost: stopping datanode
Stopping secondary namenodes [0.0.0.0]
0.0.0.0: stopping secondarynamenode
15/09/19 17:27:15 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
